In [1]:
 # Dependencies and Setup
import pandas as pd
import requests
import gmaps
import config

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
 # 1. Import the WeatherPy_database.csv file. 
df = pd.read_csv("output.csv")
df.head()

,Unnamed: 0,City,Countries,Temp,Humidity,Pressure,Latitude,Longitude
0,0,Ushuaia,AR,42.46,81,991,-54.8000,-68.3000
1,1,Bubaque,GW,76.50,77,1011,11.2833,-15.8333
2,2,Yellowknife,CA,27.12,74,1000,62.4560,-114.3525
3,3,Port Blair,IN,86.04,79,1006,11.6667,92.7500
4,4,Chui,UY,56.34,92,1021,-33.6971,-53.4616


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  47 non-null     int64  
 1   City        47 non-null     object 
 2   Countries   47 non-null     object 
 3   Temp        47 non-null     float64
 4   Humidity    47 non-null     int64  
 5   Pressure    47 non-null     int64  
 6   Latitude    47 non-null     float64
 7   Longitude   47 non-null     float64
dtypes: float64(3), int64(3), object(2)
memory usage: 3.1+ KB


In [4]:
df.describe()

,Unnamed: 0,Temp,Humidity,Pressure,Latitude,Longitude
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,23.000000,55.756383,73.978723,1012.148936,23.125891,-4.100502
std,13.711309,20.333508,15.492620,10.984136,39.695134,105.300073
min,0.000000,13.550000,33.000000,969.000000,-54.800000,-173.300000
25%,11.500000,43.085000,66.500000,1007.500000,-5.403700,-84.451000
50%,23.000000,56.340000,78.000000,1013.000000,30.415100,-15.208200
75%,34.500000,75.750000,82.500000,1020.000000,60.718400,95.764600
max,46.000000,91.510000,93.000000,1030.000000,71.916700,178.416700


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("enter a min temp "))
min_temp

enter a min temp 72


72

In [9]:
max_temp = int(input("enter a max temp "))
max_temp

enter a max temp 80


80

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
mask = (df.Temp >= min_temp) & (df.Temp <= max_temp)
df2 = df.loc[mask]
df2

,Unnamed: 0,City,Countries,Temp,Humidity,Pressure,Latitude,Longitude
1,1,Bubaque,GW,76.50,77,1011,11.2833,-15.8333
11,11,Funtua,NG,78.60,57,1009,11.5233,7.3081
14,14,Rikitea,PF,76.28,75,1017,-23.1203,-134.9692
26,26,Kapaa,US,77.43,80,1019,22.0752,-159.3190
27,27,Yaan,NG,78.49,80,1009,7.3833,8.5667
31,31,Moree,AU,75.22,43,1022,-29.4667,149.8500
46,46,Hilo,US,76.42,78,1019,19.7297,-155.0900


In [11]:
 # 4a. Determine if there are any empty rows.
df2.dropna(how="any")

,Unnamed: 0,City,Countries,Temp,Humidity,Pressure,Latitude,Longitude
1,1,Bubaque,GW,76.50,77,1011,11.2833,-15.8333
11,11,Funtua,NG,78.60,57,1009,11.5233,7.3081
14,14,Rikitea,PF,76.28,75,1017,-23.1203,-134.9692
26,26,Kapaa,US,77.43,80,1019,22.0752,-159.3190
27,27,Yaan,NG,78.49,80,1009,7.3833,8.5667
31,31,Moree,AU,75.22,43,1022,-29.4667,149.8500
46,46,Hilo,US,76.42,78,1019,19.7297,-155.0900


In [36]:
 # 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = df2[["City", "Countries", "Temp", "Humidity", "Latitude", "Longitude"]].copy().reset_index(drop=True)

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df.head(10)

,City,Countries,Temp,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
0,Bubaque,GW,76.50,77,11.2833,-15.8333,,
1,Funtua,NG,78.60,57,11.5233,7.3081,,
2,Rikitea,PF,76.28,75,-23.1203,-134.9692,,
3,Kapaa,US,77.43,80,22.0752,-159.3190,,
4,Yaan,NG,78.49,80,7.3833,8.5667,,
5,Moree,AU,75.22,43,-29.4667,149.8500,,
6,Hilo,US,76.42,78,19.7297,-155.0900,,


In [37]:
# geocoordinates
target_coordinates = "11.2833, -15.8333"
# target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

print(response.status_code)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

200
Casa Dora, Bubaque
Rua comercial Ilha de Bubaque Bubaque, Arquipélago Bijagos


In [38]:
# 6 More info for the API call
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

hotel_names = []
hotel_addresses = []

for indx, row in hotel_df.iterrows():
    print(row.City)
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    
    params["location"] = target_coordinates
    
    # make the request
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # don't forget error handling!
    if (response.status_code == 200):
        places_data = response.json()
        
        if (len(places_data["results"]) > 0):
            hotel_names.append(places_data["results"][0]["name"])
            hotel_addresses.append(places_data["results"][0]["vicinity"])
        else:
            hotel_names.append("")
            hotel_addresses.append("")
    else:
        hotel_names.append("")
        hotel_addresses.append("")
        
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Address"] = hotel_addresses
hotel_df.head(10)

Bubaque
Funtua
Rikitea
Kapaa
Yaan
Moree
Hilo


,City,Countries,Temp,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
0,Bubaque,GW,76.50,77,11.2833,-15.8333,"Casa Dora, Bubaque","Rua comercial Ilha de Bubaque Bubaque, Arquipé..."
1,Funtua,NG,78.60,57,11.5233,7.3081,Sani Jikan Mallam Hotel,"G8H5+2VJ, Dandame Road, Katsina (Capital City)"
2,Rikitea,PF,76.28,75,-23.1203,-134.9692,People ThankYou,Rikitea
3,Kapaa,US,77.43,80,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort,"650 Aleka Loop, Kapaʻa"
4,Yaan,NG,78.49,80,7.3833,8.5667,,
5,Moree,AU,75.22,43,-29.4667,149.8500,Molika Springs Motel,"314 Warialda Street, Moree"
6,Hilo,US,76.42,78,19.7297,-155.0900,Hilo Hawaiian Hotel,"71 Banyan Drive, Hilo"


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df["Hotel Name"] != ""]

,City,Countries,Temp,Humidity,Latitude,Longitude,Hotel Name,Hotel Address
0,Bubaque,GW,76.50,77,11.2833,-15.8333,"Casa Dora, Bubaque","Rua comercial Ilha de Bubaque Bubaque, Arquipé..."
1,Funtua,NG,78.60,57,11.5233,7.3081,Sani Jikan Mallam Hotel,"G8H5+2VJ, Dandame Road, Katsina (Capital City)"
2,Rikitea,PF,76.28,75,-23.1203,-134.9692,People ThankYou,Rikitea
3,Kapaa,US,77.43,80,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort,"650 Aleka Loop, Kapaʻa"
5,Moree,AU,75.22,43,-29.4667,149.8500,Molika Springs Motel,"314 Warialda Street, Moree"
6,Hilo,US,76.42,78,19.7297,-155.0900,Hilo Hawaiian Hotel,"71 Banyan Drive, Hilo"


In [40]:
 # 8a. Create the output File (CSV)
output_data_file = "hotel_output.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")